In [43]:
import tkinter as tk
from tkinter import messagebox
from tkinter import simpledialog
import mysql.connector

class ShoppingListManager:
    def __init__(self, root):
        self.root = root
        self.root.title("Manajemen Daftar Belanjaan")

        self.conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="shopping_db"
        )
        self.cursor = self.conn.cursor()

        # Membuat database dan tabel jika belum ada
        self.create_database()

        self.create_widgets()

    def create_database(self):
        self.cursor.execute("CREATE DATABASE IF NOT EXISTS shopping_db")
        self.cursor.execute("USE shopping_db")
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS shopping_list (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                price DECIMAL(10, 2) NOT NULL,
                quantity INT NOT NULL,
                subtotal DECIMAL(10, 2) NOT NULL
            )
        """)

    def create_widgets(self):
        frame_add = tk.Frame(self.root)
        frame_add.pack(pady=10)

        self.label_item = tk.Label(frame_add, text="Nama Item :")
        self.label_item.grid(row=0, column=0, padx=5)

        self.entry_item = tk.Entry(frame_add)
        self.entry_item.grid(row=0, column=1, padx=5)

        self.label_price = tk.Label(frame_add, text="Harga :")
        self.label_price.grid(row=1, column=0, padx=5)

        self.entry_price = tk.Entry(frame_add)
        self.entry_price.grid(row=1, column=1, padx=5)

        self.label_quantity = tk.Label(frame_add, text="Jumlah :")
        self.label_quantity.grid(row=2, column=0, padx=5)

        self.entry_quantity = tk.Entry(frame_add)
        self.entry_quantity.grid(row=2, column=1, padx=5)

        frame_buttons = tk.Frame(self.root)
        frame_buttons.pack(pady=10)

        self.button_add = tk.Button(frame_buttons, text="Tambah Item", command=self.add_item)
        self.button_add.grid(row=0, column=0, padx=5)

        self.button_remove = tk.Button(frame_buttons, text="Hapus Item", command=self.remove_item)
        self.button_remove.grid(row=0, column=1, padx=5)

        frame_list = tk.Frame(self.root)
        frame_list.pack(pady=10)

        self.listbox = tk.Listbox(frame_list, width=50, height=10)
        self.listbox.pack(side=tk.LEFT, padx=10)

        self.scrollbar = tk.Scrollbar(frame_list)
        self.scrollbar.pack(side=tk.LEFT, fill=tk.Y)

        self.listbox.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.listbox.yview)

        self.button_display = tk.Button(self.root, text="Tampilkan Daftar Belanjaan", command=self.display_list)
        self.button_display.pack(pady=5)

        self.button_exit = tk.Button(self.root, text="Keluar", command=self.root.quit)
        self.button_exit.pack(pady=5)

        self.load_items()

    def load_items(self):
        self.listbox.delete(0, tk.END)
        self.cursor.execute("SELECT name, price, quantity, subtotal FROM shopping_list")
        rows = self.cursor.fetchall()
        self.shopping_list = [{"name": row[0], "price": row[1], "quantity": row[2], "subtotal": row[3]} for row in rows]
        for item in self.shopping_list:
            self.listbox.insert(tk.END, f"{item['name']} - {item['quantity']} x {item['price']} = {item['subtotal']}")

    def add_item(self):
        item = self.entry_item.get()
        try:
            price = float(self.entry_price.get())
            quantity = int(self.entry_quantity.get())
            if quantity == 0 or price == 0:
                raise ZeroDivisionError
            subtotal = price * quantity
        except ValueError:
            messagebox.showerror("Input Error", "Harga dan jumlah harus berupa angka. Item tidak ditambahkan.")
            return
        except ZeroDivisionError:
            messagebox.showerror("Input Error", "Inputan tidak boleh nol. Item tidak ditambahkan.")
            return
        else:
            self.cursor.execute("INSERT INTO shopping_list (name, price, quantity, subtotal) VALUES (%s, %s, %s, %s)",
                                (item, price, quantity, subtotal))
            self.conn.commit()
            self.entry_item.delete(0, tk.END)
            self.entry_price.delete(0, tk.END)
            self.entry_quantity.delete(0, tk.END)
            self.load_items()

    def remove_item(self):
        selected_item_index = self.listbox.curselection()
        if selected_item_index:
            item_index = selected_item_index[0]
            item_name = self.shopping_list[item_index]["name"]
            self.cursor.execute("DELETE FROM shopping_list WHERE name = %s", (item_name,))
            self.conn.commit()
            self.listbox.delete(item_index)
            self.load_items()
        else:
            messagebox.showerror("Selection Error", "Pilih item yang ingin dihapus.")

    def display_list(self):
        total = sum(item['subtotal'] for item in self.shopping_list)
        items_summary = "\n".join([f"{index+1}. {item['name']} - {item['quantity']} x {item['price']} = {item['subtotal']}"
                                   for index, item in enumerate(self.shopping_list)])
        messagebox.showinfo("Daftar Belanjaan",
                            f"Daftar Belanjaan:\n\n{items_summary}\n\nTotal Keseluruhan: {total}")

if __name__ == "__main__":
    root = tk.Tk()
    app = ShoppingListManager(root)
    root.mainloop()

## Revision

In [46]:
import tkinter as tk
from tkinter import messagebox
from decimal import Decimal
import mysql.connector

class ShoppingListManager:
    def __init__(self, root):
        self.root = root
        self.root.title("Manajemen Daftar Belanjaan")

        self.conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="shopping_db"
        )
        self.cursor = self.conn.cursor()

        # Membuat database dan tabel jika belum ada
        self.create_database()

        self.create_widgets()

    def create_database(self):
        self.cursor.execute("CREATE DATABASE IF NOT EXISTS shopping_db")
        self.cursor.execute("USE shopping_db")
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS shopping_list (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                price DECIMAL(10, 2) NOT NULL,
                quantity INT NOT NULL,
                subtotal DECIMAL(10, 2) NOT NULL
            )
        """)

    def create_widgets(self):
        frame_add = tk.Frame(self.root)
        frame_add.pack(pady=10)

        self.label_item = tk.Label(frame_add, text="Nama Item\t:", anchor='w')
        self.label_item.grid(row=0, column=0, padx=5, sticky='w')

        self.entry_item = tk.Entry(frame_add, justify='left')
        self.entry_item.grid(row=0, column=1, padx=5)

        self.label_price = tk.Label(frame_add, text="Harga\t\t:", anchor='w')
        self.label_price.grid(row=1, column=0, padx=5, sticky='w')

        self.entry_price = tk.Entry(frame_add, justify='left')
        self.entry_price.grid(row=1, column=1, padx=5)

        self.label_quantity = tk.Label(frame_add, text="Jumlah\t\t:", anchor='w')
        self.label_quantity.grid(row=2, column=0, padx=5, sticky='w')

        self.entry_quantity = tk.Entry(frame_add, justify='left')
        self.entry_quantity.grid(row=2, column=1, padx=5)

        frame_buttons = tk.Frame(self.root)
        frame_buttons.pack(pady=10)

        self.button_add = tk.Button(frame_buttons, text="Tambah Item", command=self.add_item)
        self.button_add.grid(row=0, column=0, padx=5)

        self.button_remove = tk.Button(frame_buttons, text="Hapus Item", command=self.remove_item)
        self.button_remove.grid(row=0, column=1, padx=5)

        frame_list = tk.Frame(self.root)
        frame_list.pack(pady=10)

        self.listbox = tk.Listbox(frame_list, width=50, height=10)
        self.listbox.pack(side=tk.LEFT, padx=10)

        self.scrollbar = tk.Scrollbar(frame_list)
        self.scrollbar.pack(side=tk.LEFT, fill=tk.Y)

        self.listbox.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.listbox.yview)

        self.button_display = tk.Button(self.root, text="Tampilkan Daftar Belanjaan", command=self.display_list)
        self.button_display.pack(pady=5)

        self.button_exit = tk.Button(self.root, text="Keluar", command=self.root.quit)
        self.button_exit.pack(pady=5)

        self.load_items()

    def load_items(self):
        self.listbox.delete(0, tk.END)
        self.cursor.execute("SELECT name, price, quantity, subtotal FROM shopping_list")
        rows = self.cursor.fetchall()
        self.shopping_list = [{"name": row[0], "price": row[1], "quantity": row[2], "subtotal": row[3]} for row in rows]
        for item in self.shopping_list:
            self.listbox.insert(tk.END, f"{item['name']} - {item['quantity']} x {item['price']} = {item['subtotal']}")

    def add_item(self):
        item = self.entry_item.get()
        try:
            price = float(self.entry_price.get())
            quantity = int(self.entry_quantity.get())
            if quantity == 0 or price == 0:
                raise ZeroDivisionError
            subtotal = price * quantity
        except ValueError:
            messagebox.showerror("Input Error", "Harga dan jumlah harus berupa angka. Item tidak ditambahkan.")
            return
        except ZeroDivisionError:
            messagebox.showerror("Input Error", "Inputan tidak boleh nol. Item tidak ditambahkan.")
            return
        else:
            self.cursor.execute("INSERT INTO shopping_list (name, price, quantity, subtotal) VALUES (%s, %s, %s, %s)",
                                (item, price, quantity, subtotal))
            self.conn.commit()
            self.entry_item.delete(0, tk.END)
            self.entry_price.delete(0, tk.END)
            self.entry_quantity.delete(0, tk.END)
            self.load_items()

    def remove_item(self):
        selected_item_index = self.listbox.curselection()
        if selected_item_index:
            item_index = selected_item_index[0]
            item_name = self.shopping_list[item_index]["name"]
            self.cursor.execute("DELETE FROM shopping_list WHERE name = %s", (item_name,))
            self.conn.commit()
            self.listbox.delete(item_index)
            self.load_items()
        else:
            messagebox.showerror("Selection Error", "Pilih item yang ingin dihapus.")

    def display_list(self):
        total = sum(item['subtotal'] for item in self.shopping_list)
        items_summary = "\n".join([f"{index+1}. {item['name']} - {item['quantity']} x {item['price']} = {item['subtotal']}"
                                   for index, item in enumerate(self.shopping_list)])

        list_window = tk.Toplevel(self.root)
        list_window.title("Daftar Belanjaan")

        label_summary = tk.Label(list_window, text=f"Daftar Belanjaan :\n===========================\n{items_summary}\n===========================\nTotal Keseluruhan : {total:.2f}")
        label_summary.pack(pady=5)


        label_amount = tk.Label(list_window, text="Jumlah Uang:")
        label_amount.pack(pady=5)

        entry_amount = tk.Entry(list_window)
        entry_amount.pack(pady=5)

        def process_payment():
            try:
                amount_paid = Decimal(entry_amount.get())
                if amount_paid < Decimal(total):
                    messagebox.showerror("Error", "Jumlah uang kurang. Masukkan kembali.")
                    entry_amount.delete(0, tk.END)
                else:
                    change = amount_paid - Decimal(total)
                    messagebox.showinfo("Kembalian", f"Kembalian Anda: {change:.2f}")
                    # Clear listbox and database
                    self.cursor.execute("DELETE FROM shopping_list")
                    self.conn.commit()
                    self.load_items()
                    list_window.destroy()
            except (ValueError, Decimal.InvalidOperation):
                messagebox.showerror("Error", "Masukkan jumlah uang yang valid.")
                entry_amount.delete(0, tk.END)

        button_frame = tk.Frame(list_window)
        button_frame.pack(pady=5)
        
        button_pay = tk.Button(button_frame, text="Bayar", command=process_payment)
        button_pay.grid(row=0, column=0, padx=5)
        
        button_back = tk.Button(button_frame, text="Kembali", command=list_window.destroy)
        button_back.grid(row=0, column=1, padx=5)


if __name__ == "__main__":
    root = tk.Tk()
    app = ShoppingListManager(root)
    root.mainloop()


## Revision2.0

In [ ]:
import tkinter as tk
from tkinter import messagebox
from decimal import Decimal
import mysql.connector

class ShoppingListManager:
    def __init__(self, root):
        self.root = root
        self.root.title("Manajemen Daftar Belanjaan")

        self.conn = mysql.connector.connect(
            host="localhost",
            user="root",
            password="",
            database="shopping_db"
        )
        self.cursor = self.conn.cursor()

        # Membuat database dan tabel jika belum ada
        self.create_database()

        self.create_widgets()

    def create_database(self):
        self.cursor.execute("CREATE DATABASE IF NOT EXISTS shopping_db")
        self.cursor.execute("USE shopping_db")
        self.cursor.execute("""
            CREATE TABLE IF NOT EXISTS shopping_list (
                id INT AUTO_INCREMENT PRIMARY KEY,
                name VARCHAR(255) NOT NULL,
                price DECIMAL(10, 2) NOT NULL,
                quantity INT NOT NULL,
                subtotal DECIMAL(10, 2) NOT NULL
            )
        """)

    def create_widgets(self):
        frame_add = tk.Frame(self.root)
        frame_add.pack(pady=10)

        self.label_item = tk.Label(frame_add, text="Nama Item\t:", anchor='w')
        self.label_item.grid(row=0, column=0, padx=5, sticky='w')

        self.entry_item = tk.Entry(frame_add, justify='left')
        self.entry_item.grid(row=0, column=1, padx=5)

        self.label_price = tk.Label(frame_add, text="Harga\t\t:", anchor='w')
        self.label_price.grid(row=1, column=0, padx=5, sticky='w')

        self.entry_price = tk.Entry(frame_add, justify='left')
        self.entry_price.grid(row=1, column=1, padx=5)

        self.label_quantity = tk.Label(frame_add, text="Jumlah\t\t:", anchor='w')
        self.label_quantity.grid(row=2, column=0, padx=5, sticky='w')

        self.entry_quantity = tk.Entry(frame_add, justify='left')
        self.entry_quantity.grid(row=2, column=1, padx=5)

        frame_buttons = tk.Frame(self.root)
        frame_buttons.pack(pady=10)

        self.button_add = tk.Button(frame_buttons, text="Tambah Item", command=self.add_item)
        self.button_add.grid(row=0, column=0, padx=5)

        self.button_remove = tk.Button(frame_buttons, text="Hapus Item", command=self.remove_item)
        self.button_remove.grid(row=0, column=1, padx=5)

        frame_list = tk.Frame(self.root)
        frame_list.pack(pady=10)

        self.listbox = tk.Listbox(frame_list, width=50, height=10)
        self.listbox.pack(side=tk.LEFT, padx=10)

        self.scrollbar = tk.Scrollbar(frame_list)
        self.scrollbar.pack(side=tk.LEFT, fill=tk.Y)

        self.listbox.config(yscrollcommand=self.scrollbar.set)
        self.scrollbar.config(command=self.listbox.yview)

        self.button_display = tk.Button(self.root, text="Tampilkan Daftar Belanjaan", command=self.display_list)
        self.button_display.pack(pady=5)

        self.button_exit = tk.Button(self.root, text="Keluar", command=self.root.quit)
        self.button_exit.pack(pady=5)

        self.load_items()

    def load_items(self):
        self.listbox.delete(0, tk.END)
        self.cursor.execute("SELECT name, price, quantity, subtotal FROM shopping_list")
        rows = self.cursor.fetchall()
        self.shopping_list = [{"name": row[0], "price": row[1], "quantity": row[2], "subtotal": row[3]} for row in rows]
        for item in self.shopping_list:
            self.listbox.insert(tk.END, f"{item['name']} - {item['quantity']} x {item['price']} = {item['subtotal']}")

    def add_item(self):
        item = self.entry_item.get()
        try:
            price = float(self.entry_price.get())
            quantity = int(self.entry_quantity.get())
            if quantity == 0 or price == 0:
                raise ZeroDivisionError
            subtotal = price * quantity
        except ValueError:
            messagebox.showerror("Input Error", "Harga dan jumlah harus berupa angka. Item tidak ditambahkan.")
            return
        except ZeroDivisionError:
            messagebox.showerror("Input Error", "Inputan tidak boleh nol. Item tidak ditambahkan.")
            return
        else:
            self.cursor.execute("INSERT INTO shopping_list (name, price, quantity, subtotal) VALUES (%s, %s, %s, %s)",
                                (item, price, quantity, subtotal))
            self.conn.commit()
            self.entry_item.delete(0, tk.END)
            self.entry_price.delete(0, tk.END)
            self.entry_quantity.delete(0, tk.END)
            self.load_items()

    def remove_item(self):
        selected_item_index = self.listbox.curselection()
        if selected_item_index:
            item_index = selected_item_index[0]
            item_name = self.shopping_list[item_index]["name"]
            self.cursor.execute("DELETE FROM shopping_list WHERE name = %s", (item_name,))
            self.conn.commit()
            self.listbox.delete(item_index)
            self.load_items()
        else:
            messagebox.showerror("Selection Error", "Pilih item yang ingin dihapus.")

    def display_list(self):
        total = sum(item['subtotal'] for item in self.shopping_list)
        items_summary = "\n".join([f"{index+1}. {item['name']} - {item['quantity']} x {item['price']} = {item['subtotal']}"
                                   for index, item in enumerate(self.shopping_list)])

        list_window = tk.Toplevel(self.root)
        list_window.title("Daftar Belanjaan")

        label_summary = tk.Label(list_window, text=f"Daftar Belanjaan :\n===========================\n{items_summary}\n===========================\nTotal Keseluruhan : {total:.2f}")
        label_summary.pack(pady=5)

        label_amount = tk.Label(list_window, text="Jumlah Uang:")
        label_amount.pack(pady=5)

        entry_amount = tk.Entry(list_window)
        entry_amount.pack(pady=5)

        button_frame = tk.Frame(list_window)
        button_frame.pack(pady=5)
        
        button_pay = tk.Button(button_frame, text="Bayar", command=lambda: self.process_payment(entry_amount, total, list_window))
        button_pay.grid(row=0, column=0, padx=5)
        
        button_back = tk.Button(button_frame, text="Kembali", command=list_window.destroy)
        button_back.grid(row=0, column=1, padx=5)

    def process_payment(self, entry_amount, total, list_window):
        try:
            amount_paid = Decimal(entry_amount.get())
            if amount_paid < Decimal(total):
                messagebox.showerror("Error", "Jumlah uang kurang. Masukkan kembali.")
                entry_amount.delete(0, tk.END)
            else:
                change = amount_paid - Decimal(total)
                messagebox.showinfo("Kembalian", f"Kembalian Anda: {change:.2f}")

                self.listbox.delete(0, tk.END)
                # self.clear_all_items() ---> digunakan untuk memanggil function menghapus data yang telah terinput pada database
                list_window.destroy()
                
        except (ValueError, Decimal.InvalidOperation):
            messagebox.showerror("Error", "Masukkan jumlah uang yang valid.")
            entry_amount.delete(0, tk.END)

    #function untuk menghapus seluruh data yang telah terinput pada database (opsional saja)
    """def clear_all_items(self):
        # Menghapus semua item dari tabel database
        self.cursor.execute("DELETE FROM shopping_list")
        self.conn.commit()
        self.load_items()"""



if __name__ == "__main__":
    root = tk.Tk()
    app = ShoppingListManager(root)
    root.mainloop()
